## Calculate the volume integral of a given 3D permeability field using SYCL parallel computing

The problem statement solved by the given code is to calculate the volume integral of a given 3D permeability field using SYCL parallel computing.

The code first initializes a 3D permeability field of size NX x NY x NZ with values 1, 2, 3, ..., NXNYNZ using the std::iota() function. Then, it creates SYCL buffers for the permeability field and a result variable.

Next, the code submits a SYCL kernel using a parallel_for loop that calculates the volume integral of the permeability field. The kernel multiplies all the values in the permeability field together and stores the result in the result variable.

The code runs the kernel on a chosen device, either a CPU or GPU, depending on the selector that is uncommented. The device name is printed to the console to confirm which device is being used. After the kernel completes, the result of the volume integral is printed to the console.

Note that the volume integral of a 3D field is calculated by taking the product of all the values in the field. In this case, the permeability field represents the permeability of a porous medium, and the volume integral is a measure of the overall permeability of the medium.







In [1]:
%%writefile lab/simple1.cpp
#include <CL/sycl.hpp>
#include <iostream>
#include <random>

//namespace sycl = cl::sycl;
namespace my_sycl = cl::sycl;

int main() {
    constexpr size_t num_simulations = 10000;
    constexpr size_t num_years = 30;
    constexpr float initial_portfolio_value = 1000000.0f;
    constexpr float desired_withdrawal_rate = 0.04f;
    constexpr float expected_return_rate = 0.06f;
    constexpr float stock_volatility = 0.15f;
    constexpr float bond_volatility = 0.05f;
    constexpr float stock_weight = 0.7f;
    constexpr float bond_weight = 0.3f;
    constexpr float inflation_rate = 0.02f;

    std::mt19937 rng{std::random_device{}()};
    std::normal_distribution<float> stock_dist{expected_return_rate, stock_volatility};
    std::normal_distribution<float> bond_dist{expected_return_rate, bond_volatility};

    std::vector<float> portfolio_values(num_simulations, initial_portfolio_value);
    for (size_t year = 0; year < num_years; ++year) {
        for (size_t i = 0; i < num_simulations; ++i) {
            const float stock_return = stock_dist(rng);
            const float bond_return = bond_dist(rng);
            const float portfolio_return = stock_weight * stock_return + bond_weight * bond_return;
            portfolio_values[i] *= 1 + portfolio_return;
            portfolio_values[i] -= desired_withdrawal_rate * portfolio_values[i];
            portfolio_values[i] /= (1 + inflation_rate);
        }
    }

    const float success_threshold = 1.0f; // multiple of initial portfolio value
    const size_t num_successful_simulations = std::count_if(
        portfolio_values.begin(), portfolio_values.end(),
        [success_threshold](const float value) { return value >= success_threshold * initial_portfolio_value; });
    const float success_rate = static_cast<float>(num_successful_simulations) / num_simulations;
    std::cout << "Success rate: " << success_rate << std::endl;

    return 0;
}


Overwriting lab/simple1.cpp


In [3]:
! chmod 755 q; chmod 755 run_simple1.sh;if [ -x "$(command -v qsub)" ]; then ./q run_simple1.sh; else ./run_simple1.sh; fi

Job has been submitted to Intel(R) DevCloud and will execute soon.

Job ID                    Name             User            Time Use S Queue
------------------------- ---------------- --------------- -------- - -----
2262219.v-qsvr-1           ...ub-singleuser u182761         00:00:05 R jupyterhub     
2262224.v-qsvr-1           run_simple1.sh   u182761         00:00:03 R batch          
2262225.v-qsvr-1           run_simple1.sh   u182761                0 Q batch          

Waiting for Output ███████████ Done⬇

########################################################################
#      Date:           Fri 24 Mar 2023 08:57:28 AM PDT
#    Job ID:           2262225.v-qsvr-1.aidevcloud
#      User:           u182761
# Resources:           cput=75:00:00,neednodes=1:gpu:ppn=2,nodes=1:gpu:ppn=2,walltime=06:00:00
########################################################################

## u182761 is compiling DPCPP_Essentials Module1 -- oneAPI Techgig example - 1 of 1 simple1.cpp
Succe